<a href="https://colab.research.google.com/github/ChristopheLaforge/ChristopheLaforge/blob/main/Hooper2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from traitlets.traitlets import ForwardDeclaredInstance
import random
import math

class OneRandomWalk:
  implemented_graphs = ["Squared_Lattice","Triangular_Lattice",
                        "Cubic_Lattice","Hexagonal_Lattice"]
  def __init__(self,iMaximumLength,iGraph,iBeta):
    self.maximum_length = iMaximumLength
    self.graph = iGraph
    self.beta = iBeta
    self.initial_site = [0,0,0]
    self.length = 0
    self.path = [self.initial_site]
    self.winding = 0
    self.is_trapped_forward = False
    self.is_trapped_backward = False
    self.epsilon = 1
    if not self.IsGraphImplemented(): print("Graph is not implemented")

# functions that should not evolved anymore
  def IsGraphImplemented(self):
    output = False
    if self.graph in self.implemented_graphs: output = True
    return output

  def ResetWalk(self):
    self.length = 0
    self.path = [self.initial_site]
    self.is_trapped_forward = False
    self.is_trapped_backward = False
    return None

  def Atmosphere(self,iSite):
    output = len(self.OccupiedSites(self.FirstNeighbors(iSite)))
    return output

  def UnoccupiedSites(self,iListSites):
    output = [site for site in iListSites if site not in self.path]
    return output
  
  def OccupiedSites(self,iListSites):
    output = [site for site in iListSites if site in self.path]
    return output

  def FirstNeighborsInSquaredLattice(self,iSite):
    output = []
    output.append([iSite[0]+1,iSite[1],iSite[2]])
    output.append([iSite[0],iSite[1]+1,iSite[2]])
    output.append([iSite[0]-1,iSite[1],iSite[2]])
    output.append([iSite[0],iSite[1]-1,iSite[2]])
    return output

  def FirstNeighborsInCubicLattice(self,iSite):
    output = []
    output.append([iSite[0]+1,iSite[1],iSite[2]])
    output.append([iSite[0],iSite[1]+1,iSite[2]])
    output.append([iSite[0]-1,iSite[1],iSite[2]])
    output.append([iSite[0],iSite[1]-1,iSite[2]])
    output.append([iSite[0],iSite[1],iSite[2]-1])
    output.append([iSite[0],iSite[1],iSite[2]+1])
    return output

  def FirstNeighborsInTriangularLattice(self,iSite):
    output = []
    output.append([iSite[0]+1,iSite[1],iSite[2]])
    output.append([iSite[0],iSite[1]+1,iSite[2]])
    output.append([iSite[0]-1,iSite[1],iSite[2]])
    output.append([iSite[0],iSite[1]-1,iSite[2]])
    output.append([iSite[0]+1,iSite[1]+1,iSite[2]])
    output.append([iSite[0]-1,iSite[1]-1,iSite[2]])
    return output

  def FirstNeighborsInHexagonalLattice(self,iSite):
    output = []
    parity = (iSite[0]+iSite[1])%2
    output.append([iSite[0]+(2*parity-1),iSite[1],iSite[2]])
    output.append([iSite[0],iSite[1]-1,iSite[2]])
    output.append([iSite[0],iSite[1]+1,iSite[2]])
    return output

# functions that could still evolve
  def FirstNeighbors(self,iSite):
    if self.graph == "Squared_Lattice":
      output = self.FirstNeighborsInSquaredLattice(iSite)
    if self.graph == "Cubic_Lattice":
      output = self.FirstNeighborsInCubicLattice(iSite)
    if self.graph == "Triangular_Lattice":
      output = self.FirstNeighborsInTriangularLattice(iSite)
    if self.graph == "Hexagonal_Lattice":
      output = self.FirstNeighborsInHexagonalLattice(iSite)
    return output

  def SelectedMove(self,iListSites):

    # I need first to compute the atmosphere of the possible sites
    atmospheres = [self.Atmosphere(site)-1 for site in iListSites]
    free_energies = [atmosphere*self.epsilon for atmosphere in atmospheres]
    unnormalised_probabilities = [math.exp(free_energy*self.beta)
                                  for free_energy in free_energies]

    partition_function = sum(unnormalised_probabilities)

    normalised_probabilities = [probability/partition_function
                                for probability in unnormalised_probabilities]
    output = random.choices(iListSites,weights=normalised_probabilities,k=1)
    
    return output[0]

  def DoubleGrow(self):
    does_grow_forward_continue = True
    does_grow_backward_continue = True
    if self.is_trapped_forward: does_grow_forward_continue = False
    if self.is_trapped_backward: does_grow_backward_continue = False
    if self.length >= self.maximum_length: 
      does_grow_backward_continue = False
      does_grow_forward_continue = False

    while does_grow_forward_continue or does_grow_backward_continue:

      self.GrowWalkByOneStepForward()
      if self.is_trapped_forward: does_grow_forward_continue = False
      if self.length >= self.maximum_length: 
        does_grow_forward_continue = False
        does_grow_backward_continue = False
      
      self.GrowWalkByOneStepBackward()
      if self.is_trapped_backward: does_grow_backward_continue = False
      if self.length >= self.maximum_length: 
        does_grow_forward_continue = False
        does_grow_backward_continue = False
      
    return None

  def Grow(self):
    does_grow_continue = True
    if self.is_trapped_forward: does_grow_continue = False
    if self.length >= self.maximum_length: does_grow_continue = False
    while does_grow_continue:
      self.GrowWalkByOneStepForward()
      if self.is_trapped_forward: does_grow_continue = False
      if self.length >= self.maximum_length: does_grow_continue = False
    return None

  def GrowWalkByOneStepBackward(self):
    current_site = self.path[0]
    unoccupied_sites = self.UnoccupiedSites(self.FirstNeighbors(current_site))
    if len(unoccupied_sites)==0: self.is_trapped_backward= True
    if not self.is_trapped_backward:
      self.path.insert(0,self.SelectedMove(unoccupied_sites))
      self.length += 1
    return None

  def GrowWalkByOneStepForward(self):
    current_site = self.path[-1]
    unoccupied_sites = self.UnoccupiedSites(self.FirstNeighbors(current_site))
    if len(unoccupied_sites)==0: self.is_trapped_forward= True
    if not self.is_trapped_forward:
      self.path.append(self.SelectedMove(unoccupied_sites))
      self.length += 1
    return None



In [2]:
class RandomWalks:
  def __init__(self,iMaxLength,iLattice,iBeta):
    self.maximum_length = iMaxLength
    self.lattice = iLattice
    self.beta = iBeta
    self.mean_length = 0

  def Generate(self,iNbrWalks):
    walk = OneRandomWalk(self.maximum_length,self.lattice,self.beta)
    for i in range(iNbrWalks):
      walk.Grow()
      self.mean_length += walk.length
      walk.ResetWalk()
    self.mean_length = self.mean_length/iNbrWalks

  def AverageLength(self):
    return self.mean_length

In [8]:
Sample = RandomWalks(10000,"Hexagonal_Lattice",0)
Sample.Generate(100000)
Sample.AverageLength()

71.09426

In [79]:
Walk = RandomWalk(10,"Squared_Lattice")
Walk.Grow()
print(Walk.path)
Walk.length

[[0, 0], [-1, 0], [-2, 0], [-3, 0], [-3, 1], [-3, 2], [-3, 3], [-4, 3], [-5, 3], [-5, 4], [-5, 5]]


10